In [1]:
%env CUDA_VISIBLE_DEVICES="0"
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
from audiolm_pytorch import EncodecWrapper

encodec = EncodecWrapper()
import sys
sys.path.append('..')

env: CUDA_VISIBLE_DEVICES="0"


2023-11-17 07:01:36.411991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 07:01:36.597607: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--------------------------------------------------------------------------
detected:

  Local host:            150-230-187-81
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4126

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_o

In [2]:
import glob
glob.glob("/home/ubuntu/LJSpeech/pflowtts_pytorch/logs/train/ljspeech/runs/2023-11-16_16-49-44/checkpoints/*")

['/home/ubuntu/LJSpeech/pflowtts_pytorch/logs/train/ljspeech/runs/2023-11-16_16-49-44/checkpoints/last.ckpt',
 '/home/ubuntu/LJSpeech/pflowtts_pytorch/logs/train/ljspeech/runs/2023-11-16_16-49-44/checkpoints/checkpoint_epoch=019.ckpt']

In [3]:
import datetime as dt
from pathlib import Path

import IPython.display as ipd
import numpy as np
import soundfile as sf
import torch
from tqdm.auto import tqdm

from pflow.models.pflow_tts import pflowTTS
from pflow.text import sequence_to_text, text_to_sequence
from pflow.utils.model import denormalize
from pflow.utils.utils import get_user_data_dir, intersperse

from pflow.hifigan.config import v1
from pflow.hifigan.denoiser import Denoiser
from pflow.hifigan.env import AttrDict
from pflow.hifigan.models import Generator as HiFiGAN

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# This allows for real time code changes being reflected in the notebook, no need to restart the kernel

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device = "cpu"

In [6]:
get_user_data_dir()

PosixPath('/home/ubuntu/.local/share/pflow_tts')

## Filepaths

In [7]:
PFLOW_CHECKPOINT = "/home/ubuntu/LJSpeech/pflowtts_pytorch/logs/train/ljspeech/runs/2023-11-16_16-49-44/checkpoints/checkpoint_epoch=019.ckpt" #fill in the path to the pflow checkpoint
HIFIGAN_CHECKPOINT = get_user_data_dir() / "hifigan_T2_v1"
OUTPUT_FOLDER = "synth_output"

## Load TTS model

In [8]:
def load_model(checkpoint_path):
    model = pflowTTS.load_from_checkpoint(checkpoint_path, map_location=device)
    model.eval()
    return model
count_params = lambda x: f"{sum(p.numel() for p in x.parameters()):,}"


model = load_model(PFLOW_CHECKPOINT)
print(f"Model loaded! Parameter count: {count_params(model)}")

Model loaded! Parameter count: 33,515,009


In [9]:
def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

# vocoder = load_vocoder(HIFIGAN_CHECKPOINT)
# denoiser = Denoiser(vocoder, mode='zeros')

### Helper functions to synthesise

In [26]:
@torch.inference_mode()
def process_text(text: str):
    x = torch.tensor(intersperse(text_to_sequence(text, ['english_cleaners2']), 0),dtype=torch.long, device=device)[None]
    x_lengths = torch.tensor([x.shape[-1]],dtype=torch.long, device=device)
    x_phones = sequence_to_text(x.squeeze(0).tolist())
    return {
        'x_orig': text,
        'x': x,
        'x_lengths': x_lengths,
        'x_phones': x_phones
    }


@torch.inference_mode()
def synthesise(text, prompt):
    text_processed = process_text(text)
    start_t = dt.datetime.now()
    output = model.synthesise(
        text_processed['x'], 
        text_processed['x_lengths'],
        n_timesteps=n_timesteps,
        temperature=temperature,
        length_scale=length_scale,
        prompt=prompt
    )
    # merge everything to one dict    
    output.update({'start_t': start_t, **text_processed})
    return output

# @torch.inference_mode()
# def to_waveform(mel, vocoder):
#     audio = vocoder(mel).clamp(-1, 1)
#     audio = denoiser(audio.squeeze(0), strength=0.00025).cpu().squeeze()
#     return audio.cpu().squeeze()
    
# def save_to_folder(filename: str, output: dict, folder: str):
#     folder = Path(folder)
#     folder.mkdir(exist_ok=True, parents=True)
#     np.save(folder / f'{filename}', output['mel'].cpu().numpy())
#     sf.write(folder / f'{filename}.wav', output['waveform'], 22050, 'PCM_24')

## Setup text to synthesise

In [11]:
texts = [
    "text is text, one two three."
]

### Hyperparameters

In [12]:
## Number of ODE Solver steps
n_timesteps = 10

## Changes to the speaking rate
length_scale=1.0

## Sampling temperature
temperature = 0.667

In [13]:
import torchaudio
import glob
wav_files = glob.glob("/home/ubuntu/LJSpeech/LJSpeech-1.1/wavs/*.wav")
wav, sr = torchaudio.load(wav_files[0])
mel = encodec(wav, curtail_from_left = True, return_encoded = True)

In [29]:
import torchaudio
wav, sr = torchaudio.load(wav_files[0])
from pflow.data.text_mel_datamodule import mel_spectrogram
mel = mel_spectrogram(
            wav,
            1024,
            80,
            22050,
            256,
            1024,
            0,
            8000,
            center=False,
        )

In [35]:
mel = encodec(wav, curtail_from_left = True, return_encoded = True)

In [37]:
mel[0].shape

torch.Size([1, 293, 128])

In [39]:
prompt = mel[0].transpose(1,2) #load a mel spectrogram from a file and paste it here; check dimensions [batch, channels, time]

In [ ]:
from einops import rearrange
prompt = encodec.model.encoder(rearrange(wav, f'b t -> b {encodec.model.channels} t'))[0]

## Synthesis

In [ ]:
outputs, rtfs = [], []
rtfs_w = []
for i, text in enumerate(tqdm(texts)):
    prompt = prompt[:,:,:264]
    output = synthesise(text, prompt) #, torch.tensor([15], device=device, dtype=torch.long).unsqueeze(0))
    # output['waveform'] = to_waveform(output['mel'], vocoder)

    # Compute Real Time Factor (RTF) with HiFi-GAN
    # t = (dt.datetime.now() - output['start_t']).total_seconds()
    # rtf_w = t * 22050 / (output['waveform'].shape[-1])

    # ## Pretty print
    # print(f"{'*' * 53}")
    # print(f"Input text - {i}")
    # print(f"{'-' * 53}")
    # print(output['x_orig'])
    # print(f"{'*' * 53}")
    # print(f"Phonetised text - {i}")
    # print(f"{'-' * 53}")
    # print(output['x_phones'])
    # print(f"{'*' * 53}")
    # print(f"RTF:\t\t{output['rtf']:.6f}")
    # print(f"RTF Waveform:\t{rtf_w:.6f}")
    # rtfs.append(output['rtf'])
    # rtfs_w.append(rtf_w)

    # Display the synthesised waveform
    audio = encodec.decode(output['decoder_outputs'].transpose(-1,1))
    ipd.display(ipd.Audio(audio.detach().cpu().numpy()[0][0], rate=22050))

    ## Save the generated waveform
#     save_to_folder(i, output, OUTPUT_FOLDER)

# print(f"Number of ODE steps: {n_timesteps}")
# print(f"Mean RTF:\t\t\t\t{np.mean(rtfs):.6f} ± {np.std(rtfs):.6f}")
# print(f"Mean RTF Waveform (incl. vocoder):\t{np.mean(rtfs_w):.6f} ± {np.std(rtfs_w):.6f}")

  0%|          | 0/1 [00:00<?, ?it/s]